In [47]:
from lxml import html
import requests
import time

headers = {'User-Agent': 'Yandex/1.01.001'}

page = requests.get('https://www.avito.ru/volgograd/avtomobili/geely_emgrand_ec7_2013_877004308',headers=headers)
tree = html.fromstring(page.content)

def deleteLiteralsFromNumberValues(s):
    if s[0].isdigit():
        return ''.join([i for i in s if i.isdigit() or i in ['.',',']])
    else:
        return s

attributes = tree.xpath('//li[@class="item-params-list-item"]/span/text()') 
values = tree.xpath('//li[@class="item-params-list-item"]/text()')
values2 = values[1::2]
values2 = map(lambda s: deleteLiteralsFromNumberValues(s.replace('\n', '').replace(" ","").replace(u'\xa0',' ')), values2)
zipList = list(zip(attributes, values2))
zipList

[('Марка: ', 'Geely'),
 ('Модель: ', 'EmgrandEC7'),
 ('Год выпуска: ', '2013'),
 ('Пробег: ', '45000'),
 ('Тип кузова: ', 'хетчбэк'),
 ('Цвет: ', 'серебряный'),
 ('Объём двигателя: ', '1.5'),
 ('Коробка передач: ', 'механика'),
 ('Тип двигателя: ', 'бензин'),
 ('Привод: ', 'передний'),
 ('Руль: ', 'левый'),
 ('Состояние: ', 'не битый'),
 ('Владельцев по ПТС: ', '1'),
 ('Мощность двигателя: ', '98..')]

In [43]:
#цена машины
price = tree.xpath('//span[@class="price-value-string"]/text()')[0]
price = price.replace(u'\xa0',' ').replace(u'\u2009',' ').replace(u'\n',' ').replace(u' ','')
price

'360000'

In [39]:
#описание машины от пользователя
description = tree.xpath('//div[@class="item-description-text"]/p/text()') 
description

['Продается автомобиль Джили Эмгрант, 2013 г.в., состояние автомобиля отличное, вложений не требует, полностью обслужен, все время эксплуатации в одних руках, есть сервисная книжка, два комплекта ключей, комплектация полная',
 '-КОНДИЦИОНЕР',
 '-АБС',
 '-ГУР',
 '-ПЭП',
 '-кожаный салон',
 '-музыка',
 'Автомобиль можно оформить в КРЕДИТ на хороших условиях',
 'ТОРГ, ОБМЕН',
 '______________________________________________________',
 'осмотр ул. Азизбекова 70, Гарантия юридической чистоты']

In [69]:
#парсим все сслыки на географические области
mainAvtomobPage = requests.get('https://www.avito.ru/rossiya/transport')
tree = html.fromstring(mainAvtomobPage.content)

#regions = tree.xpath('//div[@class="catalog-counts__more"]/text()')[0]
regions = tree.xpath('//a[@class="js-catalog-counts__link"]')
reg = list(map(lambda x: x.get("href").replace(u"transport","avtomobili") , regions))[5:]
reg

['/moskva/avtomobili',
 '/moskovskaya_oblast/avtomobili',
 '/sankt-peterburg/avtomobili',
 '/leningradskaya_oblast/avtomobili',
 '/bashkortostan/avtomobili',
 '/belgorodskaya_oblast/avtomobili',
 '/volgogradskaya_oblast/avtomobili',
 '/voronezhskaya_oblast/avtomobili',
 '/kaliningradskaya_oblast/avtomobili',
 '/krasnodarskiy_kray/avtomobili',
 '/nizhegorodskaya_oblast/avtomobili',
 '/novosibirskaya_oblast/avtomobili',
 '/omskaya_oblast/avtomobili',
 '/permskiy_kray/avtomobili',
 '/rostovskaya_oblast/avtomobili',
 '/samarskaya_oblast/avtomobili',
 '/saratovskaya_oblast/avtomobili',
 '/sverdlovskaya_oblast/avtomobili',
 '/stavropolskiy_kray/avtomobili',
 '/tatarstan/avtomobili',
 '/tyumenskaya_oblast/avtomobili',
 '/udmurtiya/avtomobili',
 '/chelyabinskaya_oblast/avtomobili',
 '/yaroslavskaya_oblast/avtomobili']

In [79]:
allMachines = requests.get('https://www.avito.ru/volgogradskaya_oblast/avtomobili')
tree = html.fromstring(allMachines.content)

regions = tree.xpath('//a[@class="item-description-title-link"]')
ak = list(map(lambda x: x.get("href"), regions))
last_page = tree.xpath('//a[@class="pagination-page"]')[-1]
last_page = last_page.get("href")
numOfIterationScrapping = int(last_page.split("=")[-1])
numOfIterationScrapping

266

In [ ]:
#/volgogradskaya_oblast/avtomobili?p=2